<a href="https://colab.research.google.com/github/fabiobasson/Bi-Master/blob/main/geological_image_similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
from os import getcwd
import zipfile
import shutil
import numpy as np
import glob
import random
import pandas as pd
import matplotlib.pyplot as plt


from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split

**Importar arquivos de configuração do Kaglle**

In [3]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"fabiobasson","key":"d71cbfdfd23379909173204eac74cf96"}'}

In [4]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

###  Download dos dados / descompactando os dados

In [5]:
!kaggle datasets download tanyadayanand/geological-image-similarity
! unzip -qq geological-image-similarity

 51% 17.0M/33.6M [00:00<00:00, 46.3MB/s]
100% 33.6M/33.6M [00:00<00:00, 96.6MB/s]


### Data Preprocessing

In [4]:
andesite_dir = glob.glob('geological_similarity/andesite/*.jpg');gneiss_dir = glob.glob('geological_similarity/gneiss/*.jpg')
marble_dir = glob.glob('geological_similarity/marble/*.jpg');quartzite_dir = glob.glob('geological_similarity/quartzite/*.jpg')
rhyolite_dir = glob.glob('geological_similarity/rhyolite/*.jpg');schist_dir = glob.glob('geological_similarity/schist/*.jpg')
print(len(andesite_dir));print(len(gneiss_dir)); print(len(marble_dir)); print(len(quartzite_dir));  print(len(schist_dir))


5000
5000
4998
5000
5000


In [39]:
!rm -rf new_geological_similarity

In [24]:
! mkdir new_geological_similarity
!cp -r geological_similarity/andesite/* new_geological_similarity
! cp -r geological_similarity/gneiss/* new_geological_similarity


In [35]:
#geological_similarity = 'geological_similarity'
#subfolders = [f.path for f in os.scandir(geological_similarity) if f.is_dir()]

In [36]:
subfolders

['geological_similarity/gneiss',
 'geological_similarity/quartzite',
 'geological_similarity/marble',
 'geological_similarity/andesite',
 'geological_similarity/rhyolite',
 'geological_similarity/schist']

In [14]:
for i in andesite_dir[0:100]:
    andesite_dir.append([i])
for j in gneiss_dir[0:100]:
   gneiss_dir.append([j])
df = andesite_dir + gneiss_dir
#random.shuffle(df)
df

['geological_similarity/andesite/9FUK8.jpg',
 'geological_similarity/andesite/20XR8.jpg',
 'geological_similarity/andesite/PT0WZ.jpg',
 'geological_similarity/andesite/Z7KXW.jpg',
 'geological_similarity/andesite/AVLAI.jpg',
 'geological_similarity/andesite/F2ZBZ.jpg',
 'geological_similarity/andesite/3VFJB.jpg',
 'geological_similarity/andesite/U5K4D.jpg',
 'geological_similarity/andesite/7YPFT.jpg',
 'geological_similarity/andesite/3PTNB.jpg',
 'geological_similarity/andesite/G0NAI.jpg',
 'geological_similarity/andesite/X63G8.jpg',
 'geological_similarity/andesite/7TDZL.jpg',
 'geological_similarity/andesite/6HCS3.jpg',
 'geological_similarity/andesite/8B8UB.jpg',
 'geological_similarity/andesite/J0XSR.jpg',
 'geological_similarity/andesite/UYP1K.jpg',
 'geological_similarity/andesite/32KW4.jpg',
 'geological_similarity/andesite/PF1YF.jpg',
 'geological_similarity/andesite/LDEFO.jpg',
 'geological_similarity/andesite/47W7V.jpg',
 'geological_similarity/andesite/S4VQK.jpg',
 'geologic

In [38]:
class_names = ['andesite', 'gneiss', 'marbler', 'quartzite', 'rhyolite',
               'schist']

In [17]:
data_df = pd.DataFrame(df, columns = ['name'])
data_df

,name
0,geological_similarity/andesite/9FUK8.jpg
1,geological_similarity/andesite/20XR8.jpg
2,geological_similarity/andesite/PT0WZ.jpg
3,geological_similarity/andesite/Z7KXW.jpg
4,geological_similarity/andesite/AVLAI.jpg
...,...
10995,[geological_similarity/gneiss/ABJBS.jpg]
10996,[geological_similarity/gneiss/KNMTL.jpg]
10997,[geological_similarity/gneiss/06JLG.jpg]
10998,[geological_similarity/gneiss/VBEON.jpg]
